<a href="https://colab.research.google.com/github/thibaud-perrin/vision-language-models/blob/main/notebooks/vlm_usage_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processing Images and Text with Visual-Language Models (VLMs)

This notebook demonstrates how to use the `HuggingFaceTB/SmolVLM-Instruct` 4bit-quantized model for various multimodal tasks. These tasks include answering questions about images, recognizing and interpreting text in images, and analyzing videos through sequential frame analysis. By structuring prompts effectively, this notebook showcases how to apply the model to practical applications like scene understanding, document analysis, and video reasoning.

---

## What's Inside

### Processing Images
This section explores how the model processes single and multiple images for captioning and answering questions.

1. **Single Image Captioning**  
   Generate captions and answer questions about individual images. This demonstrates the model's ability to understand and describe visual content.

2. **Comparing Multiple Images**  
   Analyze and compare multiple images to identify common themes or differences. This highlights the model's capacity for visual reasoning across images.

3. **Text Recognition (OCR)**  
   Leverage the model's ability to recognize and interpret text within images, making it useful for tasks like document analysis and text-heavy visuals. We experiment with images that contain dense text to evaluate the model's OCR capabilities.



## Secrets
Loading HuggingFace secret and login to huggingFace

In [1]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [2]:
# Authenticate to Hugging Face
from huggingface_hub import login

login(token=HF_TOKEN)

## Libraries

In [3]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub accelerate bitsandbytes opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
import cv2
import numpy as np
import torch, PIL
from IPython.display import Image, display, Video
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig
from transformers.image_utils import load_image

## Load the model

In [5]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "HuggingFaceTB/SmolVLM-Instruct"
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    quantization_config=quantization_config,
).to(device)
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")

print(processor.image_processor.size)

config.json:   0%|          | 0.00/7.32k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.49G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/3.52M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/92.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.07k [00:00<?, ?B/s]

Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


{'longest_edge': 1536}


## Processing Images
This section explores how the model processes single and multiple images for captioning and answering questions.


### **Single Image Captioning**  
Generate captions and answer questions about individual images. This demonstrates the model's ability to understand and describe visual content.


In [9]:
image_url1 = "https://cdn.pixabay.com/photo/2024/11/20/09/14/christmas-9210799_1280.jpg"
display(Image(url=image_url1, height=200))

image_url2 = "https://cdn.pixabay.com/photo/2024/11/23/08/18/christmas-9218404_1280.jpg"
display(Image(url=image_url2, height=200))

In [10]:
# Load  one image
image1 = load_image(image_url1)

# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Can you describe the image?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(device)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])


/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


User:<image>Can you describe the image?
Assistant: The image is a scene of a person walking in a forest. The person is wearing a coat and a cap. The person is holding the hand of another person. The person is walking on a path. The path is covered with dry leaves. The background of the image is a forest with trees.


### **Comparing Multiple Images**  
Analyze and compare multiple images to identify common themes or differences. This highlights the model's capacity for visual reasoning across images.


In [11]:

# Load images
image2 = load_image(image_url2)

# Create input messages
messages = [
    # {
    #     "role": "user",
    #     "content": [
    #         {"type": "image"},
    #         {"type": "image"},
    #         {"type": "text", "text": "Can you describe the two images?"}
    #     ]
    # },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "What event do they both represent?"}
        ]
    },
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(device)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])

User:<image>What event do they both represent?
Assistant: Christmas.


### **Text Recognition (OCR)**  
Leverage the model's ability to recognize and interpret text within images, making it useful for tasks like document analysis and text-heavy visuals. We experiment with images that contain dense text to evaluate the model's OCR capabilities.


In [12]:
document_image_url = "https://cdn.pixabay.com/photo/2020/11/30/19/23/christmas-5792015_960_720.png"
display(Image(url=document_image_url, height=200))

# Load the document image
document_image = load_image(document_image_url)

# Create input message for analysis
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "What is written?"}
        ]
    }
]

# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[document_image], return_tensors="pt")
inputs = inputs.to(device)

# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts[0])

User:<image>What is written?
Assistant: MERRY CHRISTMAS AND A HAPPY NEW YEAR.


## Processing Videos
Visual-Language Models (VLMs) can analyze videos by extracting and processing keyframes in temporal order. Although these models lack dedicated temporal modeling, they can:
- Describe actions or events by analyzing sampled frames sequentially.
- Answer questions about videos based on representative keyframes.
- Summarize video content by combining textual descriptions of multiple frames.

In this section, we experiment with a sample video, showcasing how the model can reason about dynamic visual content using static frames.


Let experiment on an example:

<video width="640" height="360" controls>
  <source src="https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [13]:
def extract_frames(video_path, max_frames=50, target_size=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, max_frames, dtype=int)

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if target_size:
                frames.append(resize_and_crop(frame, target_size))
            else:
                frames.append(frame)
    cap.release()
    return frames

def resize_and_crop(image, target_size):
    width, height = image.size
    scale = target_size / min(width, height)
    image = image.resize((int(width * scale), int(height * scale)), PIL.Image.Resampling.LANCZOS)
    left = (image.width - target_size) // 2
    top = (image.height - target_size) // 2
    return image.crop((left, top, left + target_size, top + target_size))

# Video link
video_link = "https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4"

In [14]:
question = "Describe what the woman is doing."

def generate_response(model, processor, frames, question):

    image_tokens = [{"type": "image"} for _ in frames]
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": "Following are the frames of a video in temporal order."}, *image_tokens, {"type": "text", "text": question}]
        }
    ]
    inputs = processor(
        text=processor.apply_chat_template(messages, add_generation_prompt=True),
        images=frames,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs, max_new_tokens=100, num_beams=5, temperature=0.7, do_sample=True, use_cache=True
    )
    return processor.decode(outputs[0], skip_special_tokens=True)


# Extract frames from the video
frames = extract_frames(video_link, max_frames=15, target_size=384)

processor.image_processor.size = (384, 384)
processor.image_processor.do_resize = False
# Generate response
response = generate_response(model, processor, frames, question)

# Display the result
# print("Question:", question)
print("Response:", response)

Response: User: Following are the frames of a video in temporal order.<image>Describe what the woman is doing.
Assistant: The woman is hanging an ornament on a Christmas tree.
